# How to Train Bert For Q&A in Any Language

In [1]:
pip install -qqq tqdm datasets transformers torch

In [27]:
from tqdm.auto import tqdm  # for showing progress bar
from datasets import load_dataset
from transformers import TrainingArguments, Trainer, DefaultDataCollator
from transformers import BertTokenizerFast, BertTokenizer, BertForQuestionAnswering
from transformers import AdamW
import torch
import pandas as pd

In [3]:
squad = load_dataset('squad')

  0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [16]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [17]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [18]:
squad_train = squad['train'].select(range(1000))

In [19]:
squad_val = squad['validation'].select(range(1000))

In [20]:
tokenized_squad_train = squad_train.map(preprocess_function, batched=True, remove_columns=squad_train.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
tokenized_squad_val = squad_val.map(preprocess_function, batched=True, remove_columns=squad_val.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [25]:
data_collator = DefaultDataCollator()

Create Model

In [28]:
training_args = TrainingArguments(
    output_dir="bert-base-uncased-finetuned-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [30]:
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [32]:
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=tokenized_squad_train,
                  eval_dataset=tokenized_squad_val,
                  tokenizer=tokenizer,
                  data_collator=data_collator)

In [33]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 315


Epoch,Training Loss,Validation Loss
1,No log,3.529932
2,No log,2.543425
3,No log,2.358139
4,No log,2.287331
5,No log,2.264768


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=315, training_loss=2.505734592013889, metrics={'train_runtime': 468.9536, 'train_samples_per_second': 10.662, 'train_steps_per_second': 0.672, 'total_flos': 979862837760000.0, 'train_loss': 2.505734592013889, 'epoch': 5.0})